In [75]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [76]:
## Load the ANN trained model, scaler pickle, OHE
model = load_model('model.h5')

#load the encoder adn scaler
with open('onehot.pkl','rb') as file:
    onehot=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [ ]:
## Example input data

input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male', 
    'Age' : 40, 
    'Tenure' : 3, 
    "Balance":60000, 
    'NumOfProducts':2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
 
}

In [78]:
print(type(onehot))

<class 'sklearn.preprocessing._encoders.OneHotEncoder'>


In [79]:

geo_encoded = onehot.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns= onehot.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\siddh\Desktop\NEW_Course\Project\ANN_Comparision\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [80]:
ip_df= pd.DataFrame([input_data])
ip_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [81]:
## Encode categorical Variables
ip_df['Gender'] = label_encoder_geo.transform(ip_df['Gender'])
ip_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [82]:
## Contatination with OHE data and label_encode data
ip_df=pd.concat([ip_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
ip_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [85]:
## Scaling the data
ip_scaled =  scaler.transform(ip_df)
ip_scaled

array([[-0.55012981,  0.89091075,  0.12136034, -0.68041201, -0.28051905,
         0.84584804,  0.62670381,  0.968496  , -0.91572441,  1.00400803,
        -0.57427105, -0.58350885]])

In [93]:
## Prediction churn
prediction = model.predict(ip_scaled)
prediction

1/1 [==============================] - 0s 22ms/step


array([[0.05721828]], dtype=float32)

In [99]:
predict_prob = prediction[0][0]
predict_prob

0.05721828

In [100]:
if predict_prob > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')


The customer is not likely to churn
